In [1]:
import torch
import os
import random
import os
import numpy as np
import logging
from config import Config
from model import TorchModel, choose_optimizer
from evaluate import Evaluator
from loader import load_data2
#[DEBUG, INFO, WARNING, ERROR, CRITICAL]
logging.basicConfig(level=logging.INFO, format = '%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

"""
模型训练主程序
"""

/opt/conda/lib/python3.8/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


/opt/conda/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32
[2024-07-28 18:27:59,147] [INFO] [real_accelerator.py:133:get_accelerator] Setting ds_accelerator to cuda (auto detect)


'\n模型训练主程序\n'

In [2]:
config = Config
#创建保存模型的目录
if not os.path.isdir(config["model_path"]):
    os.mkdir(config["model_path"])
#加载训练数据
train_data = load_data2(config["train_data_path"], config)
#加载模型
model = TorchModel(config)
# 标识是否使用gpu
cuda_flag = torch.cuda.is_available()
if cuda_flag:
    logger.info("gpu可以使用，迁移模型至gpu")
    model = model.cuda()

Some weights of the model checkpoint at /mnt/workspace/.cache/modelscope/langboat/mengzi-bert-base were not used when initializing BertModel: ['sop.cls.weight', 'cls.predictions.decoder.weight', 'sop.cls.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
#加载优化器
optimizer = choose_optimizer(config, model)
#加载效果测试类
evaluator = Evaluator(config, model, logger)
#训练
for epoch in range(config["epoch"]):
    epoch += 1
    model.train()
    logger.info("epoch %d begin" % epoch)
    train_loss = []
    for index, batch_data in enumerate(train_data):
        if cuda_flag:
            batch_data = [d.cuda() for d in batch_data]

        optimizer.zero_grad()
        input_ids, labels = batch_data   #输入变化时这里需要修改，比如多输入，多输出的情况
        loss = model(input_ids, labels)
        loss.backward()
        optimizer.step()

        train_loss.append(loss.item())
        if index % int(len(train_data) / 2) == 0:
            logger.info("batch loss %f" % loss)
    logger.info("epoch average loss: %f" % np.mean(train_loss))
    acc = evaluator.eval(epoch)
    print(acc)

2024-07-28 18:28:03,638 - __main__ - INFO - epoch 1 begin
2024-07-28 18:28:53,202 - __main__ - INFO - batch loss 1.049476
